# Test of Predictive Analysis via RL

In [41]:
#Yahoo finaince not working any longer
#from yahoo_finance import Share
from enum import Enum

import matplotlib.pyplot as plt
import numpy as np
import random
import datetime
import tensorflow as tf
from tensorflow.python.framework import ops

plt.rcParams['figure.figsize'] = [26, 16]

In [2]:
class Column(Enum):
    DATE = 0
    OPEN = 1
    HIGH = 2
    LOW = 3
    CLOSE = 4
    ADJCLOSE = 5
    VOLUME = 6


In [48]:
start_date_global = '2000-07-01'
end_date_global = '2017-07-01'
#stock_name = 'BTC-USD'
#stock_name = 'ETH-USD'
stock_name = 'MSFT'
arrow_size = 10

In [29]:
def getStockDataVec(key, column, start_date=None, end_date=None):
    vec = []
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    
    lines = open("data/" + key + ".csv", "r").read().splitlines()
    for line in lines[1:]:
        tval = line.split(",")
        #Date,Open,High,Low,Close,Adj Close,Volume
        cur_date = datetime.datetime.strptime(tval[0], "%Y-%m-%d")
        if((start_date != None and end_date!=None and cur_date >= start_date and cur_date <= end_date) or
          (start_date == None and end_date == None)):
            if column.value == 0:
                vec.append(str(tval[column.value]))
            elif column.value == 1 or column.value == 2 or column.value == 3 or column.value == 4 or column.value == 5:
                vec.append(float(tval[column.value]))
            elif column.value == 6:
                vec.append(int(tval[column.value]))
        
    return vec

In [28]:
#btcusd = getStockDataVec(stock_name,Column.CLOSE)
#print(btcusd[:])

In [32]:
btcusd_dates = getStockDataVec(stock_name,Column.DATE, start_date_global, end_date_global)
btcusd_dates

['2000-07-03',
 '2000-07-05',
 '2000-07-06',
 '2000-07-07',
 '2000-07-10',
 '2000-07-11',
 '2000-07-12',
 '2000-07-13',
 '2000-07-14',
 '2000-07-17',
 '2000-07-18',
 '2000-07-19',
 '2000-07-20',
 '2000-07-21',
 '2000-07-24',
 '2000-07-25',
 '2000-07-26',
 '2000-07-27',
 '2000-07-28',
 '2000-07-31',
 '2000-08-01',
 '2000-08-02',
 '2000-08-03',
 '2000-08-04',
 '2000-08-07',
 '2000-08-08',
 '2000-08-09',
 '2000-08-10',
 '2000-08-11',
 '2000-08-14',
 '2000-08-15',
 '2000-08-16',
 '2000-08-17',
 '2000-08-18',
 '2000-08-21',
 '2000-08-22',
 '2000-08-23',
 '2000-08-24',
 '2000-08-25',
 '2000-08-28',
 '2000-08-29',
 '2000-08-30',
 '2000-08-31',
 '2000-09-01',
 '2000-09-05',
 '2000-09-06',
 '2000-09-07',
 '2000-09-08',
 '2000-09-11',
 '2000-09-12',
 '2000-09-13',
 '2000-09-14',
 '2000-09-15',
 '2000-09-18',
 '2000-09-19',
 '2000-09-20',
 '2000-09-21',
 '2000-09-22',
 '2000-09-25',
 '2000-09-26',
 '2000-09-27',
 '2000-09-28',
 '2000-09-29',
 '2000-10-02',
 '2000-10-03',
 '2000-10-04',
 '2000-10-

In [44]:
def plot_prices(prices, buytrigger=None, selltrigger=None):
    plt.title('Opening Crypto Prices')
    plt.xlabel('iteration')
    plt.ylabel('price ($)')
    plt.plot(prices)
    
    if buytrigger.size > 0:
        plt.plot(buytrigger[:,0], buytrigger[:,1],'^', markersize=arrow_size, color='g')
    if selltrigger.size > 0:
        plt.plot(selltrigger[:,0], selltrigger[:,1],'v', markersize=arrow_size, color='r')
    plt.savefig('prices.png')

## Setup our ABC for our policies

In [7]:
class DecisionPolicy:
    def select_action(self, current_state, step):
        pass
    
    def update_q(self, state, action, reward, next_state):
        pass

## Allow us to just do a simple random policy which will allow for a baseline comparison for our model later when comparing to QLearning approach and others
#### Randomly picks an 'action' without looking at the 'state'

In [8]:
class RandomDecisionPolicy(DecisionPolicy):
    def __init__(self, actions):
        self.actions = actions
        
    def select_action(self, current_state, step):
        action = self.actions[random.randint(0, len(self.actions) - 1)]
        return action

## Go about setting up a QLearning decision policy
#### Will allow us to setup a learned / more robust decision policy via TF

In [51]:
class QLearningDecisionPolicy(DecisionPolicy):
    def __init__(self, actions, input_dim):
        self.epsilon = 0.5
        self.gamma = 0.001
        self.actions = actions
        output_dim = len(actions)
        h1_dim = 200

        self.x = tf.placeholder(tf.float32, [None, input_dim])
        self.y = tf.placeholder(tf.float32, [output_dim])
        W1 = tf.Variable(tf.random_normal([input_dim, h1_dim]))
        b1 = tf.Variable(tf.constant(0.1, shape=[h1_dim]))
        h1 = tf.nn.relu(tf.matmul(self.x, W1) + b1)
        W2 = tf.Variable(tf.random_normal([h1_dim, output_dim]))
        b2 = tf.Variable(tf.constant(0.1, shape=[output_dim]))
        self.q = tf.nn.relu(tf.matmul(h1, W2) + b2)

        loss = tf.square(self.y - self.q)
        self.train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
        self.sess = tf.Session()
        self.sess.run(tf.initialize_all_variables())

    def select_action(self, current_state, step):
        threshold = min(self.epsilon, step / 1000.)
        if random.random() < threshold:
            # Exploit best option with probability epsilon
            action_q_vals = self.sess.run(self.q, feed_dict={self.x: current_state})
            action_idx = np.argmax(action_q_vals)  # TODO: replace w/ tensorflow's argmax
            action = self.actions[action_idx]
        else:
            # Explore random option with probability 1 - epsilon
            action = self.actions[random.randint(0, len(self.actions) - 1)]
        return action

    def update_q(self, state, action, reward, next_state):
        action_q_vals = self.sess.run(self.q, feed_dict={self.x: state})
        next_action_q_vals = self.sess.run(self.q, feed_dict={self.x: next_state})
        next_action_idx = np.argmax(next_action_q_vals)
        action_q_vals[0, next_action_idx] = reward + self.gamma * next_action_q_vals[0, next_action_idx]
        action_q_vals = np.squeeze(np.asarray(action_q_vals))
        self.sess.run(self.train_op, feed_dict={self.x: state, self.y: action_q_vals})


In [10]:
def run_simulation(policy, initial_budget, initial_num_stocks, prices, hist, num_try=0, debug=False):
    budget = initial_budget
    num_stocks = initial_num_stocks
    share_value = 0
    transitions = list()
    buytriggers = list()
    selltriggers = list()
    for i in range(len(prices) - hist - 1):
        if i % 100 == 0:
            print('try {} - progress {:.2f}%'.format(num_try, float(100*i) / (len(prices) - hist - 1)))
        current_state = np.asmatrix(np.hstack((prices[i:i+hist], budget, num_stocks)))
        current_portfolio = budget + num_stocks * share_value
        action = policy.select_action(current_state, i)
        share_value = float(prices[i + hist + 1])
        
        if action == 'Buy' and budget >= share_value:
            budget -= share_value
            num_stocks += 1
            buytriggers.append((i, prices[i]))
        elif action == 'Sell' and num_stocks > 0:
            budget += share_value
            num_stocks -= 1
            selltriggers.append((i, prices[i]))
        else: 
            action = 'Hold'
        new_portfolio = budget + num_stocks * share_value
        reward = new_portfolio - current_portfolio
        next_state = np.asmatrix(np.hstack((prices[i+1:i+hist+1], budget, num_stocks)))
        transitions.append((current_state, action, reward, next_state))
        policy.update_q(current_state, action, reward, next_state)
        
    portfolio = budget + num_stocks * share_value
    if debug:
        print('${}\t{} shares'.format(budget, num_stocks))
    return portfolio, np.array(buytriggers), np.array(selltriggers)

## Run our model a number of times to then get the best mean across the runs which will provide a more reliable result

In [25]:
def run_simulations(policy, budget, num_stocks, prices, hist):
    num_tries = 100
    final_portfolios = list()
    for i in range(num_tries):
        final_portfolio, _, __ = run_simulation(policy, budget, num_stocks, prices, hist, i)
        final_portfolios.append(final_portfolio)
    avg, std = np.mean(final_portfolios), np.std(final_portfolios)
    return avg, std, _, __

In [52]:
if __name__ == '__main__':
    #prices = getStockDataVec(stock_name, Column.OPEN, '2015-01-01','2016-01-01')
    prices = getStockDataVec(stock_name, Column.OPEN, start_date_global, end_date_global)
    
    actions = ['Buy', 'Sell', 'Hold']
    hist = 200#len(prices) #Last number of stock prices (Size of each state)
    #policy = RandomDecisionPolicy(actions)
    policy = QLearningDecisionPolicy(actions, hist + 2)
    budget = 1000.0
    num_stocks = 0
    
    #portfolio, buytrig, selltrig = run_simulation(policy, budget, num_stocks, prices, hist)
    avg, std, buytrig, selltrig = run_simulations(policy, budget, num_stocks, prices, hist)
    plot_prices(prices, buytrig, selltrig)
    print(avg, std)

try 0 - progress 0.00%


TypeError: list indices must be integers or slices, not Tensor